In [ ]:
import importlib
from __imports__ import *
import data, LSTM_returns, optimization
importlib.reload(data)

Assets = pd.DataFrame(index=['df', 'column', 'LSTM', 'train_X_y'], columns=['SOL-30m', 'SOL-6h', 'SOL-1d'], dtype='object')

Assets.at['df', 'SOL-30m'] = data.load_asset('SOLUSDT', sampling='30m')
Assets.at['df', 'SOL-6h'] = data.load_asset('SOLUSDT', sampling='6h')
Assets.at['df', 'SOL-1d'] = data.load_asset('SOLUSDT', sampling='1d')

for key in Assets.columns:
    df = Assets.loc['df', key].copy()
    df = data.subset(df, start=pd.Timestamp('2021-01-01'), end=pd.Timestamp('2026-01-01'))
    df = data.train_test_split(df)
    df = data.add_fear_and_greed(df, includeCategory=False)
    Assets.at['df', key] = data.add_returns(df)

    target = "Return_Target"
    features = ["Open", "High", "Low", "Close", "Volume", "F&G"]
    Assets.at['train_X_y', key] = LSTM_returns.vectorize_train_data(df, target=target, features=features, episode_length=30)

AttributeError: 'float' object has no attribute 'copy'

In [ ]:
hyperparams = {}

for key in Assets.columns:
    LSTM = LSTM_returns.create(hyperparams)
    history = LSTM_returns.train(LSTM, hyperparams, Assets.loc['train_X_y', key])
    LSTM_returns.all_folds_plot(LSTM, Assets.loc['train_X_y', key])

    Assets.at['LSTM', key] = LSTM

0    2
1    4
2    6
Name: a, dtype: int64